In [ ]:
import pandas as pd
from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27017/")
db = client["Bank_2025"]
collection = db["prestamos"]

df = pd.read_csv('../database/bancos_clean.csv')
data = df.to_dict(orient="records")
collection.insert_many(data)
print("Datos insertados en MongoDB correctamente.")
## ✅ A. Mostrar todos los documentos
# Imprimir cada documento completo (cada fila del CSV convertido a JSON) almacenado en la colección préstamos.
print("Visualizar datos insertados en MongoDB.")
for doc in collection.find():
    print(doc)
## ✅ B. Mostrar solo los primeros registros
# Ideal para no saturar la consola.
for doc in collection.find({}, {"_id": 0, "num_cta": 1, "saldo": 1}):
    print(doc)
## ✅ C. Consultar solo algunos campos
# Por ejemplo, mostrar solo el número de cuenta y saldo:
# El parámetro {"_id": 0} oculta el campo _id autogenerado por MongoDB.
# Se muestran únicamente num_cta y saldo.
for doc in collection.find({}, {"_id": 0, "Num_Cta": 1, "Saldo": 1}):
    print(doc)
## ✅ D. Consultar con filtro (condición WHERE)
# Por ejemplo, mostrar cuentas con saldo mayor a 10,000:
#  $gt significa “greater than” (mayor que).
#  También existen otros operadores: $lt, $gte, $lte, $ne, $in, etc.
consulta = {"saldo": {"$gt": 10000}}
for doc in collection.find(consulta, {"_id": 0, "Cli_codigo": 1, "Saldo": 1}):
    print(doc)
## ✅ E. Contar cuántos documentos hay en total
print("Total de registros:", collection.count_documents({}))
## ✅ F. Buscar por sucursal específica
consulta = {"Sucursal": "Cajamarca"}
for doc in collection.find(consulta):
    print(doc)
## ✅ G. Consultar solo clientes con un tipo de cuenta específico
consulta = {"TipoCuenta": "CTA AHORROS SOLES"}
for doc in collection.find(consulta):
    print(doc)
